In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [15]:
import numpy as np

np.set_printoptions(suppress=True, precision=4)  # Muestra sin notación científica y con 4 decimales
print(residuals)

[[-1. ]
 [ 0.5]
 [ 0.5]
 [-0. ]
 [-0. ]]


Nota: Para este ejemplo asumiremos que los datos de entrenamiento son sobre los que trabajaremos

In [6]:
Y = np.array([[3], [1], [8], [3], [5]])
Y

array([[3],
       [1],
       [8],
       [3],
       [5]])

In [7]:
X = np.array ([[1,3,5], [1,1,4],[1,5,6], [1,2,4], [1,4,6]])
X

array([[1, 3, 5],
       [1, 1, 4],
       [1, 5, 6],
       [1, 2, 4],
       [1, 4, 6]])

In [8]:
XT_X = np.matmul(np.matrix.transpose(X), X)
XT_X

array([[  5,  15,  25],
       [ 15,  55,  81],
       [ 25,  81, 129]])

In [10]:
XT_X_inv = np.linalg.inv(XT_X)
XT_X_inv

array([[26.7,  4.5, -8. ],
       [ 4.5,  1. , -1.5],
       [-8. , -1.5,  2.5]])

In [11]:
XT_Y = np.matmul(np.matrix.transpose(X), Y)
XT_Y

array([[ 20],
       [ 76],
       [109]])

In [12]:
betas = np.matmul(XT_X_inv, XT_Y)
betas

array([[ 4. ],
       [ 2.5],
       [-1.5]])

In [ ]:
# Calculo para los pronosticos para Y de acuerdo a los coeficientes de regresión
y_pred = np.matmul(X, betas)
y_pred

array([[4. ],
       [0.5],
       [7.5],
       [3. ],
       [5. ]])

In [16]:
# calculo de los residuos
residuals = Y - y_pred
residuals

array([[-1. ],
       [ 0.5],
       [ 0.5],
       [-0. ],
       [-0. ]])

In [18]:
#calculo de la suma de los residuos al cuadrado
RSS = float(np.matmul(np.matrix.transpose(residuals), residuals))
RSS

1.4999999999999991

In [19]:
# calculo de la suma de los cuadrados totales
TSS = float(np.matmul(np.matrix.transpose(Y - np.mean(Y)), Y - np.mean(Y)))
TSS

28.0

In [20]:
# calculo del coeficiente de determinación R^2
R_squared = float(1 - (RSS / TSS))
R_squared

0.9464285714285715

In [22]:
# calculo de coeficiente de determinación ajustado R^2
RsAdj = float(1 - (RSS / (X.shape[0] - X.shape[1])) / (TSS / (X.shape[0] - 1)))
RsAdj

0.8928571428571429

In [26]:
# calculo de la varianza del error de regresion
s_cuad = RSS / (len(Y) - X.shape[1])
s_cuad

0.7499999999999996

In [27]:
#desviación estándar del error de regresión
import math
s = math.sqrt(s_cuad)
s

0.8660254037844384

In [ ]:
# calculo de las t´s estadísticas para coeficientes de regresión

#crear ciclo para calcular las t's estadísticas
result_t = []
for i in range(0,X.shape[1]):
    t = float(betas[i] / (s * math.sqrt(XT_X_inv[i] [i])))
    result_t.append(t)
result_t

[0.893868697538675, 2.8867513459481344, -1.0954451150103264]

Criterio 1:

In [30]:
# obtener valor critico de t de la tablas
import scipy.stats

grados_libertad = len(Y) - X.shape[1]
t_critico = abs(scipy.stats.t.ppf(q=0.025,df= grados_libertad))# 95% de confianza
t_critico

np.float64(4.302652729696144)

In [33]:
for i in range(0, X.shape[1]):
    if abs(result_t[i]) > t_critico:
        print(f"Beta {i+1} es significativo") # aqui se rechaza la hipótesis nula
    else:
        print(f"Beta {i+1} no es significativo") # aqui no se rechaza la hipótesis nula

Beta 1 no es significativo
Beta 2 no es significativo
Beta 3 no es significativo


Criterio 2

In [34]:
# calculo de valores de p para las t's estadísticas

for i  in range(0, X.shape[1]):
    print("Valor p de Beta", i,":", scipy.stats.t.sf(abs(result_t[i]), df=grados_libertad) * 2)  # multiplicamos por 2 para obtener el valor p de dos colas

Valor p de Beta 0 : 0.4657159826085253
Valor p de Beta 1 : 0.10197348986612516
Valor p de Beta 2 : 0.38762756430420753


Si manejamos un nivel alfa del 5%, en niguno de los casos el valor p es menor al 5%, por lo que no podemos rechazar H0

Criterio 3

In [35]:
# calculo de intervalos de confianza del 95% para el verdadero valor de los coeficientes de regresión

for i in range(0, X.shape[1]):
    print("El valor de Beta", i, "se encuentra entre:", float(betas[i]) - t_critico * s * math.sqrt(XT_X_inv[i][i]),
          "y", float(betas[i]) + t_critico * s * math.sqrt(XT_X_inv[i][i]))

El valor de Beta 0 se encuentra entre: -15.25407049846963 y 23.254070498469808
El valor de Beta 1 se encuentra entre: -1.226206567579316 y 6.226206567579329
El valor de Beta 2 se encuentra entre: -7.39164989291438 y 4.391649892914396


Conclusion: Ninguna de las variables regresoras (independientes) es significativamente diferente de cero

In [37]:
# Comparación de resultados contra reporte automatizado de statsmodels
import statsmodels.api as sm

regresor =sm.OLS(Y, X).fit()
print(regresor.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.946
Model:                            OLS   Adj. R-squared:                  0.893
Method:                 Least Squares   F-statistic:                     17.67
Date:                Mon, 23 Jun 2025   Prob (F-statistic):             0.0536
Time:                        18:15:25   Log-Likelihood:                -4.0848
No. Observations:                   5   AIC:                             14.17
Df Residuals:                       2   BIC:                             13.00
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.0000      4.475      0.894      0.4

d:\Github\Benmaza-1\Benmaza\Lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 5 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


In [40]:
# variable para realizar dataframe y realizar correlacion

data =pd.DataFrame(X)
data2 = data.iloc[:,1:3]
data2.corr()

,1,2
1,1.000000,0.948683
2,0.948683,1.000000


In [ ]:
# Eliminación de una columna de la matriz X
X_nueva = np.delete(X, 2, 1) 
X_nueva

array([[1, 3],
       [1, 1],
       [1, 5],
       [1, 2],
       [1, 4]])

In [42]:
regresor = sm.OLS(Y, X_nueva).fit()
print(regresor.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.886
Method:                 Least Squares   F-statistic:                     32.00
Date:              lun., 23 jun. 2025   Prob (F-statistic):             0.0109
Time:                        18:31:24   Log-Likelihood:                -5.2598
No. Observations:                   5   AIC:                             14.52
Df Residuals:                       3   BIC:                             13.74
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.8000      0.938     -0.853      0.4

d:\Github\Benmaza-1\Benmaza\Lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 5 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


In [ ]:
i